# Challenge: Spacenet6 - Multi-Sensor All-Weather Mapping

This notebook demonstrates the usage of the winning [solution](https://github.com/SpaceNetChallenge/SpaceNet_SAR_Buildings_Solutions/tree/master/1-zbigniewwojna) of the [Spacenet6 challenge](https://spacenet.ai/sn6-challenge/). 

The goal there was to automatically extract building footprints with computer vision and artificial intelligence (AI) algorithms using a combination of SAR and electro-optical imagery datasets. 

For that purpose the organisers have collected over 120 sq km of both high resolution synthetic aperture radar (SAR) data and electro optical (EO) imagery with ~48,000 building footprint labels of Rotterdam, The Netherlands.

The winning solution of the challenge was implemented in `aitlas`. This notebook will demonstrate how to use it.

In [1]:
from aitlas.datasets import SpaceNet6Dataset

As a first step we need to download and prepare the data set. The instructions for downloading are available at the main [page](https://spacenet.ai/sn6-challenge/) of the challenge. Basically, you just need an AWS account with an access key and secret access key in your local configuration file. The data set is quite large (40 GB) and it takes a while to download. Once you do that, initialize a `SpaceNet6Dataset` object and call the `prepare` method to create training folds:

In [2]:
### Parameters

# base directory after unzipping the downloaded files from S3
ROOT_DIRECTORY = "/Users/spopov/train/AOI_11_Rotterdam"

# directory for storing the ground-truth segmentation masks
SEGMENTATION_DIRECTORY = "/Users/spopov/results/segmentation_masks"

# directory for storing information about the training / validation folds
FOLDS_DIR = "/Users/spopov/results/folds"

# orientation file indicating the directions from which each SAR image is captured (0 North, 1 South).
ORIENTS = "/Users/spopov/train/AOI_11_Rotterdam/SummaryData/SAR_orientations.txt"

# CSV of building footprint locations in pixel coordinates
ORIENTS_OUTPUT = "/Users/spopov/train/AOI_11_Rotterdam/SummaryData/SAR_orientations.csv"

# directory for storing the predicted masks
PRED_FOLDER = "/Users/spopov/results/pred{}"

# path to the CSV file containing the fold splits
FOLDS_PATH = "/Users/spopov/results/folds/folds.csv"

# path to the CSV containing the predicted buildings
PRED_CSV = "/Users/spopov/results/folds/pred_csv{}.csv"

# path to the CSV containing the ground-truth buildings
GT_CSV = "/Users/spopov/results/folds/gt_fold{}.csv"

/Users/spopov/Downloads/aitlas/venv/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
raw_data_set = SpaceNet6Dataset({
    "root_directory": ROOT_DIRECTORY,
    "segmentation_directory": SEGMENTATION_DIRECTORY,
    "folds_dir": FOLDS_DIR,
    "num_threads": 12,
    "edge_width": 3,
    "contact_width": 9,
    "orients": ORIENTS,
    "orients_output": ORIENTS_OUTPUT
})

The `prepare` method on the dataset object creates the segmentation masks and the training / validation folds. After this step we can actually ignore the `raw_data_set` object and create new `SpaceNet6DataSet` objects for training and validation.

In [4]:
# raw_data_set.prepare()

In [5]:
from aitlas.models import UNetEfficientNet

In `aitlas` we have implemented the winning base model of the challenge. More information can be found in this [paper](https://proceedings.mlr.press/v97/tan19a/tan19a.pdf), in Table 2 under **Efficient Net B5**. Using this model requires installation of few packages that are not specified as requirements for `aitlas`, since they are unique to this use case. Refer to the source code of `aitlas.models.unet_efficientnet.py` for information.

In [6]:
model = UNetEfficientNet({
    "net": "b5",
    "stride": 32,
    "use_cuda": True
})

/Users/spopov/Downloads/aitlas/venv/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Creating training and validation sets.

In [7]:
training_data_set = SpaceNet6Dataset({
    "batch_size": 4,
    "shuffle": True,
    "num_workers": 8,
    "transforms": [
        "aitlas.transforms.SpaceNet6Transforms"
    ],
    "orients": ORIENTS_OUTPUT,
    "pred_folder": PRED_FOLDER,
    "folds_path": FOLDS_PATH,
    "root_directory": ROOT_DIRECTORY,
    "segmentation_directory": SEGMENTATION_DIRECTORY,
    "edge_weight": 0.25,
    "contact_weight": 0.1
})

In [8]:
validation_data_set = SpaceNet6Dataset({
    "batch_size": 1,
    "shuffle": False,
    "num_workers": 1,
    "orients": ORIENTS_OUTPUT,
    "folds_path": FOLDS_PATH,
    "root_directory": ROOT_DIRECTORY,
    "start_val_epoch": 0,
    "pred_csv": PRED_CSV,
    "gt_csv": GT_CSV
})

Lastly, we just call `train_and_evaluate_model`. This will train the model for the specified number of epochs, and after the `start_val_epoch` will evaluate its performance in terms of F1 score, where precision and recall would be calculated on a pixel level (precision is the number of pixels that are predicted and are actually buildings out of all predicted as buildings, and recall is the number of pixels that are predicted as buildings out of all pixels that are buildings). The predicted images will be stored in the `PRED_FOLDER` directory, and a CSV file of all buildings will be outputed at `PRED_CSV` in a format required by the SpaceNet6 challenge.

In [9]:
model.train_and_evaluate_model(
    train_dataset=training_data_set,
    epochs=1,
    model_directory="/Users/spopov/results/model",
    save_epochs=1,
    val_dataset=validation_data_set,
    run_id="1"
)

  0%|          | 0/694 [00:26<?, ?it/s]

  0%|          | 0/266 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/266 [00:00<?, ?it/s]